In [104]:
import pandas as pd
import numpy as np
import datetime as dt


In [105]:
train = pd.read_csv("sets_de_datos/train.csv")

In [106]:
train["fecha"] = pd.to_datetime(train["fecha"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

## Categorias contempladas hasta el momento:
- Incidencia delictiva
- Pobreza (porcentual)
- Clima
- Restaurantes

### Libros con datos por año para cada provincia:

-Anuario estadístico y geográfico por entidad federativa 2013:
http://inegi.blob.core.windows.net/sistemas/productos/bvinegi/productos/integracion/pais/aepef/2013/aegpef_2013.pdf

-Anuario estadístico y geográfico por entidad federativa 2014:
https://books.google.com.ar/books?id=MmBUDwAAQBAJ&pg=PA2022&lpg=PA2022&dq=turistas+por+entidad+federativa+a%C3%B1o+2012&source=bl&ots=nSf3PUuHHl&sig=ACfU3U3QjotaQpyoTaMQIDri7L5qE50Bcg&hl=en&sa=X&ved=2ahUKEwiPp4DCpv7lAhXCGrkGHTftBzoQ6AEwBnoECAoQAQ#v=onepage&q=superficie%20agricola&f=false

-Anuario estadístico y geográfico por entidad federativa 2015:
http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/AEGPEF_2015/702825077297.pdf

-Anuario estadístico y geográfico por entidad federativa 2015:
http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/AEGPEF_2015/702825077297.pdf

-Anuario estadístico y geográfico por entidad federativa 2016:
http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/AEGPEF_2016/702825087357.pdf

Tienen que buscar los cuadros del mismo tipo en cada libro. Algunos incluyen a 2012.

### Funciones auxiliares

In [107]:
def asignar_delincuencia_por_provincia_fecha(train, df):
    """Recibe el set de datos y un dataset adicional con el index de provincias y columnas con varios años.
    Comprueba para casa en qué año fue publicada y en qué provincia y le asigna el valor de incidencia delictiva
    para ese año y esa provincia, medido cada 100000 habitantes."""
    delincuencia = []
    train["anio"] = train["fecha"].dt.year
    for index, row in train.iterrows():
        provincia = row["provincia"]
        anio = str(row["anio"])
        incidencia = df.loc[provincia][anio]
        delincuencia.append(incidencia)
    train["delincuencia"] = delincuencia
    train.drop(["anio"], axis = 1)
    return train

def asignar_caracteristica_por_fecha(train, df, nombre_columna):
    """Recibe el set de datos y un dataset adicional con el index de provincias y columnas con varios años.
    Comprueba para casa en qué año fue publicada y en qué provincia y le asigna el valor de la columna correspondiente a
    esa provincia y año."""
    caracteristicas = []
    train["anio"] = train["fecha"].dt.year
    for index, row in train.iterrows():
        provincia = row["provincia"]
        anio = str(row["anio"])
        caracteristica = df.loc[provincia][anio]
        caracteristicas.append(caracteristica)
    train[nombre_columna] = caracteristicas
    train.drop(["anio"], axis = 1)
    return train

def rellenar_año_faltante(df):
    df["2015"].fillna(0)
    columna = []
    for index, row in df.iterrows():
        caracteristica_a = row["2014"]
        caracteristica_b = row["2016"]
        if (pd.isna(caracteristica_a) and  not pd.isna(caracteristica_b)):
            columna.append(caracteristica_b)
        elif (not pd.isna(caracteristica_a) and not pd.isna(caracteristica_b)):
            columna.append(int((int(caracteristica_a) + int(caracteristica_b)) / 2))
        else:
            columna.append(caracteristica_a)
    df["2015"] = columna
    return df

### Compruebo los NAN por columna:

In [108]:
train.isna().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [109]:
train.dropna(subset=["provincia"], inplace=True)

## INCIDENCIA DELICTIVA

In [110]:
incidencia_delictiva = pd.read_csv("data/Tasa_de_incidencia_delictiva_por_entidad_federativa_de_ocurrencia_por_cada_cien_mil_habitantes.csv")

In [111]:
incidencia_delictiva.head()

,Entidad,2010 /1,2011 /2,2012 /3,2013 /4,2014,2015,2016,2017,2018
0,Estados Unidos Mexicanos,"30,535","29,200","35,139","41,563","41,655","35,497","37,017","39,369","37,807"
1,Aguascalientes,"56,089","25,511","32,368","24,711","39,453","35,457","41,254","39,912","36,500"
2,Baja California,"31,791","29,446","39,297","57,066","56,632","32,758","51,286","43,921","42,725"
3,Baja California Sur,"25,779","28,884","31,049","23,747","34,700","25,577","29,939","25,690","28,377"
4,Campeche,"20,922","21,704","29,097","30,597","29,306","22,114","28,892","28,283","26,466"


### Compruebo las fechas máximas y mínimas de train

In [112]:
train["fecha"].max()

Timestamp('2016-12-31 00:00:00')

In [113]:
train["fecha"].min()

Timestamp('2012-01-01 00:00:00')

Me quedo sólo con los años que están en train

In [114]:
incidencia_delictiva = incidencia_delictiva[["Entidad", "2012 /3", "2013 /4", "2014", "2015", "2016"]]

### Compruebo las provincias:

In [115]:
incidencia_delictiva.Entidad.unique()

array(['Estados Unidos Mexicanos', 'Aguascalientes', 'Baja California',
       'Baja California Sur', 'Campeche', 'Coahuila de Zaragoza',
       'Colima', 'Chiapas', 'Chihuahua', 'Ciudad de México', 'Durango',
       'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'México',
       'Michoacán de Ocampo', 'Morelos', 'Nayarit', 'Nuevo León',
       'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí',
       'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala',
       'Veracruz de Ignacio de la Llave', 'Yucatán', 'Zacatecas'],
      dtype=object)

Cambio los nombres para que coincidan con train

In [116]:
incidencia_delictiva.replace({"Baja California": "Baja California Norte",
                              "Coahuila de Zaragoza": "Coahuila",
                              "Michoacán de Ocampo": "Michoacán",
                              "Veracruz de Ignacio de la Llave": "Veracruz",
                              "México": "Edo. de México",
                              "Ciudad de México": "Distrito Federal",
                              "San Luis Potosí": "San luis Potosí"}, inplace = True)

In [117]:
incidencia_delictiva.columns = ["provincia", "2012", "2013", "2014", "2015", "2016"]

In [118]:
incidencia_delictiva.head()

,provincia,2012,2013,2014,2015,2016
0,Estados Unidos Mexicanos,"35,139","41,563","41,655","35,497","37,017"
1,Aguascalientes,"32,368","24,711","39,453","35,457","41,254"
2,Baja California Norte,"39,297","57,066","56,632","32,758","51,286"
3,Baja California Sur,"31,049","23,747","34,700","25,577","29,939"
4,Campeche,"29,097","30,597","29,306","22,114","28,892"


In [119]:
incidencia_delictiva = incidencia_delictiva.set_index("provincia")
incidencia_delictiva.head()

,2012,2013,2014,2015,2016
provincia,,,,,
Estados Unidos Mexicanos,"35,139","41,563","41,655","35,497","37,017"
Aguascalientes,"32,368","24,711","39,453","35,457","41,254"
Baja California Norte,"39,297","57,066","56,632","32,758","51,286"
Baja California Sur,"31,049","23,747","34,700","25,577","29,939"
Campeche,"29,097","30,597","29,306","22,114","28,892"


In [120]:
#train["anio"] = train["fecha"].dt.year

In [121]:
train = asignar_delincuencia_por_provincia_fecha(train, incidencia_delictiva)

In [122]:
train.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,anio,delincuencia
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0,2015,"52,718"
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,-99.227655,2013-06-28,0.0,0.0,0.0,1.0,1.0,3600000.0,2013,"51,786"
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,NaN,2015-10-17,0.0,0.0,0.0,0.0,0.0,1200000.0,2015,"49,317"
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,-99.688015,2012-03-09,0.0,0.0,0.0,1.0,1.0,650000.0,2012,"56,752"
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,NaN,2016-06-07,0.0,0.0,0.0,0.0,0.0,1150000.0,2016,"41,874"


## POBREZA

In [123]:
pobreza = pd.read_csv("data/pobreza_porcentual.csv")

In [124]:
pobreza.head()

,provincia,2008,2010,2012,2014,2016,2018
0,Aguascalientes,37.6,38.1,37.8,34.8,28.2,26.2
1,Baja California,26.0,31.5,30.2,28.6,22.2,23.3
2,Baja California Sur,21.4,31.0,30.1,30.3,22.1,18.1
3,Campeche,45.9,50.5,44.7,43.6,43.8,46.2
4,Coahuila,32.7,27.8,27.9,30.2,24.8,22.5


Faltan datos de los impares. Saco el promedio entre los años contiguos para obtener su valor.

In [125]:
pobreza["2013"] = (pobreza["2012"] + pobreza["2014"]) / 2
pobreza["2015"] = (pobreza["2014"] + pobreza["2016"]) / 2

In [126]:
pobreza.head()

,provincia,2008,2010,2012,2014,2016,2018,2013,2015
0,Aguascalientes,37.6,38.1,37.8,34.8,28.2,26.2,36.30,31.5
1,Baja California,26.0,31.5,30.2,28.6,22.2,23.3,29.40,25.4
2,Baja California Sur,21.4,31.0,30.1,30.3,22.1,18.1,30.20,26.2
3,Campeche,45.9,50.5,44.7,43.6,43.8,46.2,44.15,43.7
4,Coahuila,32.7,27.8,27.9,30.2,24.8,22.5,29.05,27.5


### Compruebo las provincias:

In [127]:
pobreza_provincias = pobreza["provincia"].unique()
pobreza_provincias.sort()
pobreza_provincias

array(['Aguascalientes', 'Baja California', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Ciudad de México', 'Coahuila',
       'Colima', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Michoacán', 'Morelos', 'México', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [128]:
train_provincias = train["provincia"].unique()
train_provincias.sort()
train_provincias

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima',
       'Distrito Federal', 'Durango', 'Edo. de México', 'Guanajuato',
       'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán', 'Morelos',
       'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro',
       'Quintana Roo', 'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'],
      dtype=object)

In [129]:
pobreza.replace({"Baja California": "Baja California Norte",
                 "México": "Edo. de México",
                 "Ciudad de México": "Distrito Federal",
                 "San Luis Potosí": "San luis Potosí"}, inplace = True)

In [130]:
pobreza = pobreza.set_index("provincia")

In [131]:
train = asignar_caracteristica_por_fecha(train, pobreza, "pobreza_porcentual")

In [132]:
pobreza

,2008,2010,2012,2014,2016,2018,2013,2015
provincia,,,,,,,,
Aguascalientes,37.6,38.1,37.8,34.8,28.2,26.2,36.30,31.50
Baja California Norte,26.0,31.5,30.2,28.6,22.2,23.3,29.40,25.40
Baja California Sur,21.4,31.0,30.1,30.3,22.1,18.1,30.20,26.20
Campeche,45.9,50.5,44.7,43.6,43.8,46.2,44.15,43.70
Coahuila,32.7,27.8,27.9,30.2,24.8,22.5,29.05,27.50
Colima,27.4,34.7,34.4,34.3,33.6,30.9,34.35,33.95
Chiapas,77.0,78.5,74.7,76.2,77.1,76.4,75.45,76.65
Chihuahua,32.1,38.8,35.3,34.4,30.6,26.3,34.85,32.50
Distrito Federal,27.6,28.5,28.9,28.4,27.6,30.6,28.65,28.00


In [133]:
train

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,anio,delincuencia,pobreza_porcentual
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0,2015,"52,718",28.00
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,2013-06-28,0.0,0.0,0.0,1.0,1.0,3600000.0,2013,"51,786",28.65
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,2015-10-17,0.0,0.0,0.0,0.0,0.0,1200000.0,2015,"49,317",33.60
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,2012-03-09,0.0,0.0,0.0,1.0,1.0,650000.0,2012,"56,752",45.30
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,2016-06-07,0.0,0.0,0.0,0.0,0.0,1150000.0,2016,"41,874",31.80
5,126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,...,2014-03-18,0.0,0.0,0.0,0.0,1.0,1100000.0,2014,"59,545",28.40
6,139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,NaN,3.0,1.0,...,2016-02-23,0.0,0.0,0.0,0.0,0.0,1150000.0,2016,"27,897",70.40
7,5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.0,4.0,2.0,...,2016-10-20,0.0,0.0,0.0,0.0,0.0,4200000.0,2016,"32,862",28.80
8,44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,...,2014-01-06,0.0,0.0,0.0,1.0,1.0,310000.0,2014,"30,535",34.30
9,134537,terreno,"terreno de 5.500m2 bardeado, uso de suelo h-20...",Terreno,Av. Morelos,Ixtapaluca,Edo. de México,NaN,NaN,NaN,...,2016-12-22,0.0,0.0,0.0,0.0,0.0,6200000.0,2016,"62,751",47.90


## Clima

In [134]:
clima = pd.read_csv("data/clima.csv")

In [135]:
clima

,provincia,Intervalo de temperatura media annual (minimo),Intervalo de temperatura media annual (máximo),Seco,Templado,Cálido,Frío
0,Aguascalientes,12,22,1,1,0,0
1,Baja California Norte,6,24,1,1,0,0
2,Baja California Sur,14,26,1,1,0,0
3,Campeche,24,26,0,0,1,0
4,Coahuila,8,22,1,1,0,0
5,Colima,10,28,1,1,1,0
6,Chiapas,10,30,0,1,1,0
7,Chihuahua,8,26,1,1,1,0
8,Distrito Federal,6,18,1,1,0,0
9,Durango,8,26,1,1,1,0


### Compruebo las provincias

In [136]:
clima_provincias = clima["provincia"].unique()
clima_provincias.sort()
clima_provincias

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila ', 'Colima',
       'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Michoacán', 'Morelos', 'México', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [137]:
train_provincias

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima',
       'Distrito Federal', 'Durango', 'Edo. de México', 'Guanajuato',
       'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán', 'Morelos',
       'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro',
       'Quintana Roo', 'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'],
      dtype=object)

In [138]:
clima.replace({"México": "Edo. de México"}, inplace = True)

In [139]:
train = pd.merge(train, clima, on = "provincia", how = "left")

In [140]:
train.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,precio,anio,delincuencia,pobreza_porcentual,Intervalo de temperatura media annual (minimo),Intervalo de temperatura media annual (máximo),Seco,Templado,Cálido,Frío
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,2273000.0,2015,"52,718",28.00,6.0,18.0,1.0,1.0,0.0,0.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,3600000.0,2013,"51,786",28.65,6.0,18.0,1.0,1.0,0.0,0.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,1200000.0,2015,"49,317",33.60,4.0,28.0,1.0,1.0,1.0,0.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,650000.0,2012,"56,752",45.30,0.0,26.0,1.0,1.0,1.0,1.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,1150000.0,2016,"41,874",31.80,4.0,28.0,1.0,1.0,1.0,0.0


## TURISMO (restaurantes, discotecas, etc)

In [141]:
turismo_2012 = pd.read_csv("data/servicio_turistico_2012.csv")

In [142]:
turismo_2013 = pd.read_csv("data/servicio_turistico_2013.csv")

In [143]:
turismo_2014 = pd.read_csv("data/servicio_turistico_2014.csv")

In [144]:
turismo_2016 = pd.read_csv("data/servicio_turistico_2016.csv")

In [145]:
turismo_2012["provincia"].unique()

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
       'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Edo. de México', 'Michoacán', 'Morelos', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [146]:
turismo_2013["provincia"].unique()

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
       'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Edo. de México', 'Michoacán', 'Morelos', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [147]:
turismo_2014["provincia"].unique()

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
       'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo',
       'Jalisco', 'Edo. de México', 'Michoacán', 'Morelos', 'Nayarit',
       'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo',
       'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas',
       'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'], dtype=object)

In [148]:
turismo_2016["provincia"].unique()

array(['Aguascalientes', 'Baja California Norte', 'Baja California Sur',
       'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua',
       'Distrito Federal\t', 'Durango', 'Guanajuato', 'Guerrero',
       'Hidalgo', 'Jalisco', 'Edo. de México', 'Michocán', 'Morelos',
       'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro',
       'Quintana Roo', 'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas'],
      dtype=object)

In [149]:
turismo_2016.replace({"Distrito Federal\t": "Distrito Federal"}, inplace = True)

In [222]:
turismo_2012.head()

,provincia,total,restaurantes,cafeterias,discotecas y centros nocturnos,bares,otros
0,Aguascalientes,359,247,51,16,45,ND
1,Baja California Norte,525,361,46,35,68,15
2,Baja California Sur,388,314,20,25,29,ND
3,Campeche,807,613,59,34,101,ND
4,Coahuila,884,715,40,31,98,ND


### Restaurantes

In [238]:
restaurantes = pd.DataFrame(columns=["provincia", "2012", "2013", "2014", "2015", "2016"])

In [239]:
restaurantes["provincia"] = turismo_2012["provincia"]
restaurantes["2012"] = turismo_2012["restaurantes"]
restaurantes["2013"] = turismo_2013["restaurantes"]
restaurantes["2014"] = turismo_2014["restaurantes"]
restaurantes["2016"] = turismo_2016["restaurantes"]
restaurantes.replace({"ND": np.nan}, inplace = True)

In [241]:
restaurantes = rellenar_año_faltante(restaurantes)

In [243]:
restaurantes = restaurantes.T.fillna(restaurantes.mean(axis=1)).T

In [250]:
restaurantes = restaurantes.set_index("provincia")

In [251]:
train = asignar_caracteristica_por_fecha(train, restaurantes, "restaurantes")

In [252]:
restaurantes

,2012,2013,2014,2015,2016
provincia,,,,,
Aguascalientes,247,265,303,360,417
Baja California Norte,361,361,361,1718,1718
Baja California Sur,314,328,310,542,775
Campeche,613,607,607,610,614
Coahuila,715,715,2210,2234,2259
Colima,302,303,298,314,330
Chiapas,1265,1290,1293,1297,1301
Chihuahua,2258,2467,2631,2631,2258
Distrito Federal,1116,1074,967,943,920


## PBI

In [307]:
pbi_2012 = pd.read_csv("data/pbi_2012.csv")
pbi_2013 = pd.read_csv("data/pbi_2013.csv")
pbi_2014 = pd.read_csv("data/pbi_2014.csv")
pbi_2015 = pd.read_csv("data/pbi_2015.csv")
pbi_2016 = pd.read_csv("data/pbi_2016.csv")

In [309]:
pbi_2013

,provincia,total,pbi_campo,pbi_minería,pbi_energia_agua_gas,pbi_construccion,pbi_industrias_manufactureras,pbi_comercio
0,Aguascalientes,147075,5654,1754,1272,16736,44499,22034
1,Baja California Norte,383296,10984,1058,17941,42206,73309,62216
2,Baja California Sur,100023,3222,3326,4236,15652,2310,16136
3,Campeche,631107,3579,506913,1440,44626,2421,13369
4,Coahuila,437286,9673,11519,12297,22988,176362,49141
5,Colima,76636,3744,655,6633,9752,4633,12689
6,Chiapas,229586,17722,19007,8977,18567,19155,41384
7,Chihuahua,374805,26095,9610,12903,28630,77241,58011
8,Distrito Federal,2243141,1186,295,11744,64787,158272,370801
9,Durango,156766,15213,5648,4938,13849,30720,22795
